In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 12 17:18:03 2022

@author: Kenny
"""

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import datetime

import category_encoders as ce


from sklearn.preprocessing import StandardScaler


from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns

#%%
os.chdir(r'C:\Users\Kenny\Desktop\HW2')

df = pd.read_csv(r'train.csv')
df_test = pd.read_csv(r'test.csv')


In [2]:
#%% 處理缺失值
df = df.drop(['Employee ID'],axis=1)
df_test = df_test.drop(['Employee ID'],axis=1)

print(df.isna().sum())
print(df_test.isna().sum())

df = df[df['Burn Rate'].notna()]

df = df.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


Date of Joining           0
Gender                    0
Company Type              0
WFH Setup Available       0
Designation               0
Resource Allocation     630
Mental Fatigue Score    966
Burn Rate               484
dtype: int64
Date of Joining           0
Gender                    0
Company Type              0
WFH Setup Available       0
Designation               0
Resource Allocation     110
Mental Fatigue Score    186
dtype: int64


In [3]:
#%%
X = df[df.columns[0:-1]]
y = df[df.columns[-1]]

X_test = df_test


In [4]:
#%% 類別型資料
X_category = X[['Gender','Company Type','WFH Setup Available']]
X_test_category = X_test[['Gender','Company Type','WFH Setup Available']]

CE = ce.TargetEncoder().fit(X_category,y)
X[['Gender','Company Type','WFH Setup Available']] = CE.transform(X_category)
X_test[['Gender','Company Type','WFH Setup Available']] = CE.transform(X_test_category)


D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
D:\anaconda3\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
C:\Users\Kenny\AppData\Local\Temp\ipykernel_5456\538514056.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[['Gender','Company Type','WFH Setup Available']] = CE

In [5]:
#%%日期
X['Year of Joining'] = ''
X['Month of Joining'] = ''
X['Day of Joining'] = ''
X_test['Year of Joining'] = ''
X_test['Month of Joining'] = ''
X_test['Day of Joining'] = ''

for i in range(len(X)):
    temp_str = X.iloc[i,0]
    temp_list = temp_str.split('/')
    X['Year of Joining'][i] = int(temp_list[0])
    X['Month of Joining'][i] = int(temp_list[1])
    X['Day of Joining'][i] = int(temp_list[2])

for i in range(len(X_test)):
    temp_str = X_test.iloc[i,0]
    temp_list = temp_str.split('/')
    X_test['Year of Joining'][i] = int(temp_list[0])
    X_test['Month of Joining'][i] = int(temp_list[1])
    X_test['Day of Joining'][i] = int(temp_list[2])

X = X.drop(['Date of Joining'],axis=1)
X_test = X_test.drop(['Date of Joining'],axis=1)

X = X.drop(['Year of Joining'],axis=1)
X_test = X_test.drop(['Year of Joining'],axis=1)

X = X.drop(['Month of Joining'],axis=1)
X_test = X_test.drop(['Month of Joining'],axis=1)

X = X.drop(['Day of Joining'],axis=1)
X_test = X_test.drop(['Day of Joining'],axis=1)


C:\Users\Kenny\AppData\Local\Temp\ipykernel_5456\1183823507.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Year of Joining'][i] = int(temp_list[0])
C:\Users\Kenny\AppData\Local\Temp\ipykernel_5456\1183823507.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Month of Joining'][i] = int(temp_list[1])
C:\Users\Kenny\AppData\Local\Temp\ipykernel_5456\1183823507.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Day of J

In [6]:
#%%
from sklearn.impute import KNNImputer 
X_columns = X

imputer = KNNImputer(n_neighbors=5) 
X = imputer.fit_transform(X) 
X_test = imputer.transform(X_test) 
X = pd.DataFrame(X, columns=X_columns.columns) 
X_test = pd.DataFrame(X_test, columns=X_columns.columns)


In [7]:
#%%數值型資料
numerical_column_list = ['Designation','Resource Allocation','Mental Fatigue Score']
numerical_data_others = X[numerical_column_list]
numerical_data_others_2 = X_test[numerical_column_list]

scaler = StandardScaler().fit(numerical_data_others)
numerical_data_others = scaler.transform(numerical_data_others)
numerical_data_others_2 = scaler.transform(numerical_data_others_2)

X[numerical_column_list] = numerical_data_others
X_test[numerical_column_list] = numerical_data_others_2


In [8]:
#%%
import optuna

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def objective(trial, X=X, y=y):
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 300, 3000, 300),
        'eta': trial.suggest_float("eta", 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'gamma': trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 10),
        'grow_policy': trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0)
    }

    reg = XGBRegressor(**params)
    reg.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)], eval_metric='mae',
            verbose=False)
    return mean_absolute_error(y_valid, reg.predict(X_valid))

#%%
# Creating Optuna object and defining its parameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials = 1000)

# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)


[I 2022-10-28 01:30:31,069] A new study created in memory with name: no-name-51051c60-de16-4328-a13f-ffbc34bbc80a
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:30:37,326] Trial 0 finished with value: 0.1280204896664446 and parameters: {'max_depth': 9, 'subsample': 0.5597548443989158, 'n_estimators': 2100, 'eta': 0.0001843940382860992, 'alpha': 2.6456584333061067e-08, 'lambda': 1.823451238170829e-05, 'gamma': 0.621226052391338, 'min_child_weight': 6, 'grow_policy': 'depthwise', 'colsample_bytree': 0.3476703698368766}. Best is trial 0 with value: 0.1280204896664446.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:32:06,357] Trial 12 finished with value: 0.04849284322025217 and parameters: {'max_depth': 7, 'subsample': 0.25254317831629536, 'n_estimators': 3000, 'eta': 0.05847798136988402, 'alpha': 0.7724515606945366, 'lambda': 0.001669754941542635, 'gamma': 1.040671274643242e-06, 'min_child_weight': 8, 'grow_policy': 'lossguide', 'colsample_bytree': 0.720799115962131}. Best is trial 5 with value: 0.0484114272380296.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:32:20,215] Trial 13 finished with value: 0.052162408279728914 and parameters: {'max_depth': 13, 

[I 2022-10-28 01:33:56,527] Trial 24 finished with value: 0.14616290126510373 and parameters: {'max_depth': 14, 'subsample': 0.4939492716578757, 'n_estimators': 2100, 'eta': 7.658393784779588e-05, 'alpha': 0.003519437992677326, 'lambda': 1.8114552614468568e-06, 'gamma': 2.9785847215923815e-06, 'min_child_weight': 9, 'grow_policy': 'lossguide', 'colsample_bytree': 0.5497010173576286}. Best is trial 17 with value: 0.04822766779859086.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:34:05,495] Trial 25 finished with value: 0.0793538718208717 and parameters: {'max_depth': 10, 'subsample': 0.4172127840935704, 'n_estimators': 2700, 'eta': 0.0004321801380413781, 'alpha': 0.02706406057525723, 'lambda': 0.002057603524744951, 'gamma': 0.0004406930388711116, 'min_child_weight': 10, 'grow_policy': 'los

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:35:54,758] Trial 37 finished with value: 0.06297385810691619 and parameters: {'max_depth': 8, 'subsample': 0.7080960099827976, 'n_estimators': 2400, 'eta': 0.0005536866834541355, 'alpha': 0.006111777603884831, 'lambda': 0.28192205240643253, 'gamma': 2.8894003546452674e-05, 'min_child_weight': 4, 'grow_policy': 'depthwise', 'colsample_bytree': 0.966971154861396}. Best is trial 26 with value: 0.04713766475804808.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:36:04,831] Trial 38 finished with value: 0.04788812354129671 and parameters: {'max_depth': 

[I 2022-10-28 01:37:41,044] Trial 49 finished with value: 0.0477314707067885 and parameters: {'max_depth': 8, 'subsample': 0.4020627401164288, 'n_estimators': 3000, 'eta': 0.03243550992368153, 'alpha': 0.46932519156043173, 'lambda': 1.8750463914601977e-05, 'gamma': 0.002761789994841139, 'min_child_weight': 6, 'grow_policy': 'depthwise', 'colsample_bytree': 0.8279741097476827}. Best is trial 48 with value: 0.047035597077556834.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 01:37:51,293] Trial 50 finished with value: 0.04907099234275346 and parameters: {'max_depth': 8, 'subsample': 0.38306190590080597, 'n_estimators': 3000, 'eta': 0.03708949227055668, 'alpha': 0.37060020278598876, 'lambda': 2.5936029343222947e-05, 'gamma': 0.0028046339612217675, 'min_child_weight': 5, 'grow_policy': 'depthwise

...

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 03:05:43,677] Trial 962 finished with value: 0.047318268487527806 and parameters: {'max_depth': 4, 'subsample': 0.5629031964801614, 'n_estimators': 1200, 'eta': 0.016752376510347597, 'alpha': 0.253130312296288, 'lambda': 2.8871676720166356e-05, 'gamma': 7.921558407338763e-05, 'min_child_weight': 10, 'grow_policy': 'lossguide', 'colsample_bytree': 0.8361303106425916}. Best is trial 223 with value: 0.04452063640492542.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 03:05:46,507] Trial 963 finished with value: 0.04616917160793951 and parameters: {'max_dept

[I 2022-10-28 03:06:14,401] Trial 974 finished with value: 0.04791596743848798 and parameters: {'max_depth': 4, 'subsample': 0.7669326115634976, 'n_estimators': 900, 'eta': 0.019141434532290502, 'alpha': 0.0033452850155856035, 'lambda': 0.0002794410010017412, 'gamma': 0.00014612463458069303, 'min_child_weight': 9, 'grow_policy': 'lossguide', 'colsample_bytree': 0.8761441654705199}. Best is trial 223 with value: 0.04452063640492542.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 03:06:20,013] Trial 975 finished with value: 0.0482437111587754 and parameters: {'max_depth': 13, 'subsample': 0.7945384286736339, 'n_estimators': 1200, 'eta': 0.0101846357070816, 'alpha': 0.3646435394777779, 'lambda': 0.023421626226243566, 'gamma': 0.0002465855578891732, 'min_child_weight': 10, 'grow_policy': 'depthwi

D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 03:06:59,308] Trial 987 finished with value: 0.04907467611949805 and parameters: {'max_depth': 5, 'subsample': 0.6429868785983667, 'n_estimators': 900, 'eta': 0.06828920341410785, 'alpha': 0.14494600744124997, 'lambda': 3.772874196118911e-06, 'gamma': 0.00015879701443733012, 'min_child_weight': 7, 'grow_policy': 'lossguide', 'colsample_bytree': 0.8269476156295947}. Best is trial 223 with value: 0.04452063640492542.
D:\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-10-28 03:07:05,902] Trial 988 finished with value: 0.04801318912245604 and parameters: {'max_depth'

[I 2022-10-28 03:07:58,729] Trial 999 finished with value: 0.04768474077768401 and parameters: {'max_depth': 5, 'subsample': 0.8141765856701767, 'n_estimators': 1200, 'eta': 0.017681418488948025, 'alpha': 0.06787678864682531, 'lambda': 3.466907182952072e-06, 'gamma': 2.9748941079692452e-05, 'min_child_weight': 10, 'grow_policy': 'lossguide', 'colsample_bytree': 0.547068376526553}. Best is trial 223 with value: 0.04452063640492542.


Number of finished trials: 1000
Best trial parameters: {'max_depth': 5, 'subsample': 0.6688575030557604, 'n_estimators': 1200, 'eta': 0.004506857312534973, 'alpha': 0.18612222710979467, 'lambda': 1.0785483271666074e-05, 'gamma': 0.0006260391360123853, 'min_child_weight': 7, 'grow_policy': 'lossguide', 'colsample_bytree': 0.864647078408527}
Best score: 0.04452063640492542


In [9]:
#%%
# 建立 XGBRegressor 模型
model = XGBRegressor(**study.best_trial.params)
# 使用訓練資料訓練模型
model.fit(X, y)

#%%
from sklearn.model_selection import cross_val_score, cross_validate
# directly use cross_val_score
scores = cross_val_score(model, X, y, cv=5)
print("R2: %0.2f (std: %0.2f)" % (scores.mean(), scores.std()))
print(scores)
# need other scoring

#%%
scores = cross_validate(model, X, y, cv=5,
                       scoring=('r2', 'neg_mean_squared_error', 'neg_mean_absolute_percentage_error'))
print("====================================")
print("R2: {} (std: {})".format(scores['test_r2'].mean(), scores['test_r2'].std()))
print("MSE: {} (std: {})".format(-scores['test_neg_mean_squared_error'].mean(), scores['test_neg_mean_squared_error'].std()))
print("MAPE: {} (std: {})".format(-scores['test_neg_mean_absolute_percentage_error'].mean(), scores['test_neg_mean_absolute_percentage_error'].std()))


R2: 0.90 (std: 0.01)
[0.9131127  0.90129021 0.9075902  0.89990251 0.89264324]
R2: 0.9029077707382397 (std: 0.006972437081875153)
MSE: 0.0038255789581277374 (std: 0.0001841987958337291)
MAPE: 1954120206415.4812 (std: 458753809955.9938)


In [10]:
#%%
y_pred = model.predict(X_test)

#%%
df_ans = pd.DataFrame(y_pred.astype(float), columns=['Burn Rate'])
df_ans.to_csv('mySubmission_1.csv', index_label='Employee ID')